In [40]:
import numpy as np
import tensorflow as tf

from keras.utils import load_img, img_to_array
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.layers import Flatten, Dense, Dropout
from keras import Model

In [61]:
def init_vgg16():

    # Download the model with weights pre-trained using ImageNet database
    vgg16 = VGG16(weights='imagenet', 
                  include_top=False,
                  input_shape=(224,224,3))
    
    
    # Freeze layers for training
    for layer in vgg16.layers:
        layer.trainable = False

    # Create a new 'top' of the model of fully-connected layers
    top_model = vgg16.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu', name="top_dense_1")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(512, activation='relu', name="top_dense_2")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(256, activation='relu', name="top_dense_3")(top_model)
    top_model = Dropout(0.2)(top_model)
    top_model = Dense(128, activation='relu', name="top_dense_4")(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(1, activation='softmax', name="output" )(top_model)
    
    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=vgg16.input, outputs=output_layer)

    # Compiles the model for training.
    model.compile(optimizer='adam', 
                  loss='binary_crossentropy',
                  metrics=['accuracy'])
    
    return model


In [62]:
def load_and_preprocess_img(path: str):

    img = load_img(path, target_size=(224, 224))
    ary = img_to_array(img)
    ary = np.expand_dims(ary, axis=0)
    ary = preprocess_input(ary)
    return ary[0]


In [63]:
from os import listdir

def load_data(path: str, label: int):

    X = []
    y = []

    for filename in listdir(path):
        x = load_and_preprocess_img(path + filename)
        X.append(x)
        y.append(label)

    return X, y


In [64]:
FILEPATH_POS = "./out_pos/"
FILEPATH_NEG = "./out_neg/"

X, y = load_data(FILEPATH_POS, 1)
X_neg, y_neg = load_data(FILEPATH_NEG, 0)

X.extend(X_neg)
y.extend(y_neg)

X = np.array(X)
y = np.array(y)

print(f"X: {X.shape}")
print(f"y: {y.shape}")


X: (6767, 224, 224, 3)
y: (6767,)


In [65]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [66]:
model = init_vgg16()
model.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0   

In [67]:
model.fit(X_train, y_train, batch_size=32, epochs=1)

2023-04-18 20:36:23.254836: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2729373696 exceeds 10% of free system memory.


142/142 [==============================] - 339s 2s/step - loss: 4.0402 - accuracy: 0.9173


In [68]:
model.evaluate(X_test, y_test, batch_size=32)

70/70 [==============================] - 120s 2s/step - loss: 0.3038 - accuracy: 0.9114


[0.3037729263305664, 0.9113697409629822]